In [1]:
import json
import pandas as pd
import numpy as np
import string
import itertools
import matplotlib.pyplot as plt
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
#read data from json file under yelp-dataset/yelp_training_set
business = pd.read_json('yelp-dataset/yelp_training_set/yelp_training_set_business.json', lines=True)
checkin = pd.read_json('yelp-dataset/yelp_training_set/yelp_training_set_checkin.json', lines=True)
review = pd.read_json('yelp-dataset/yelp_training_set/yelp_training_set_review.json', lines=True)
user = pd.read_json('yelp-dataset/yelp_training_set/yelp_training_set_user.json', lines=True)

In [16]:
#rename columns: name -> business_name, stars -> business_stars
business = business.rename(columns={'name': 'business_name', 'stars': 'business_stars'})
business.info()
business.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   business_id     11537 non-null  object 
 1   full_address    11537 non-null  object 
 2   open            11537 non-null  bool   
 3   categories      11537 non-null  object 
 4   city            11537 non-null  object 
 5   review_count    11537 non-null  int64  
 6   business_name   11537 non-null  object 
 7   neighborhoods   11537 non-null  object 
 8   longitude       11537 non-null  float64
 9   state           11537 non-null  object 
 10  business_stars  11537 non-null  float64
 11  latitude        11537 non-null  float64
 12  type            11537 non-null  object 
dtypes: bool(1), float64(3), int64(1), object(8)
memory usage: 1.1+ MB


,business_id,full_address,open,categories,city,review_count,business_name,neighborhoods,longitude,state,business_stars,latitude,type
0,rncjoVoEFUJGCUoC1JgnUA,"8466 W Peoria Ave\nSte 6\nPeoria, AZ 85345",True,"[Accountants, Professional Services, Tax Servi...",Peoria,3,Peoria Income Tax Service,[],-112.241596,AZ,5.0,33.581867,business
1,0FNFSzCFP_rGUoJx8W7tJg,"2149 W Wood Dr\nPhoenix, AZ 85029",True,"[Sporting Goods, Bikes, Shopping]",Phoenix,5,Bike Doctor,[],-112.105933,AZ,5.0,33.604054,business
2,3f_lyB6vFK48ukH6ScvLHg,"1134 N Central Ave\nPhoenix, AZ 85004",True,[],Phoenix,4,Valley Permaculture Alliance,[],-112.073933,AZ,5.0,33.460526,business
3,usAsSV36QmUej8--yvN-dg,"845 W Southern Ave\nPhoenix, AZ 85041",True,"[Food, Grocery]",Phoenix,5,Food City,[],-112.085377,AZ,3.5,33.392210,business
4,PzOqRohWw7F7YEPBz6AubA,"6520 W Happy Valley Rd\nSte 101\nGlendale Az, ...",True,"[Food, Bagels, Delis, Restaurants]",Glendale Az,14,Hot Bagels & Deli,[],-112.200264,AZ,3.5,33.712797,business


In [14]:
checkin.info()
checkin.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8282 entries, 0 to 8281
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   checkin_info  8282 non-null   object
 1   type          8282 non-null   object
 2   business_id   8282 non-null   object
dtypes: object(3)
memory usage: 194.2+ KB


,checkin_info,type,business_id
0,"{'11-3': 17, '8-5': 1, '15-0': 2, '15-3': 2, '...",checkin,KO9CpaSPOoqm0iCWm5scmg
1,"{'0-5': 1, '2-6': 2, '2-5': 3, '3-6': 1, '3-5'...",checkin,oRqBAYtcBYZHXA7G8FlPaA
2,"{'13-4': 1, '7-4': 1, '15-3': 1, '18-5': 1, '2...",checkin,6cy2C9aBXUwkrh4bY1DApw
3,"{'13-5': 1, '17-6': 1, '15-1': 1, '20-0': 1, '...",checkin,D0IB17N66FiyYDCzTlAI4A
4,"{'16-2': 1, '14-5': 1, '12-5': 2, '15-4': 1, '...",checkin,HLQGo3EaYVvAv22bONGkIw


In [13]:
review.info()
review.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229907 entries, 0 to 229906
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   votes        229907 non-null  object        
 1   user_id      229907 non-null  object        
 2   review_id    229907 non-null  object        
 3   stars        229907 non-null  int64         
 4   date         229907 non-null  datetime64[ns]
 5   text         229907 non-null  object        
 6   type         229907 non-null  object        
 7   business_id  229907 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 14.0+ MB


,votes,user_id,review_id,stars,date,text,type,business_id
0,"{'funny': 0, 'useful': 5, 'cool': 2}",rLtl8ZkDX5vH5nAx9C3q5Q,fWKvX83p0-ka4JS3dc6E5A,5,2011-01-26,My wife took me here on my birthday for breakf...,review,9yKzy9PApeiPPOUJEtnvkg
1,"{'funny': 0, 'useful': 0, 'cool': 0}",0a2KyEL0d3Yb1V6aivbIuQ,IjZ33sJrzXqU-0X6U8NwyA,5,2011-07-27,I have no idea why some people give bad review...,review,ZRJwVLyzEJq1VAihDhYiow
2,"{'funny': 0, 'useful': 1, 'cool': 0}",0hT2KtfLiobPvh6cDC8JQg,IESLBzqUCLdSzSqm0eCSxQ,4,2012-06-14,love the gyro plate. Rice is so good and I als...,review,6oRAC4uyJCsJl1X0WZpVSA
3,"{'funny': 0, 'useful': 2, 'cool': 1}",uZetl9T0NcROGOyFfughhg,G-WvGaISbqqaMHlNnByodA,5,2010-05-27,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,_1QQZuf4zZOyFCvXc0o6Vg
4,"{'funny': 0, 'useful': 0, 'cool': 0}",vYmM4KTsC8ZfQBg-j5MWkw,1uJFq2r5QfJG_6ExMRCaGw,5,2012-01-05,General Manager Scott Petello is a good egg!!!...,review,6ozycU1RpktNG2-1BroVtw


In [12]:
user.info()
user.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43873 entries, 0 to 43872
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   votes          43873 non-null  object 
 1   user_id        43873 non-null  object 
 2   name           43873 non-null  object 
 3   average_stars  43873 non-null  float64
 4   review_count   43873 non-null  int64  
 5   type           43873 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.0+ MB


,votes,user_id,name,average_stars,review_count,type
0,"{'funny': 0, 'useful': 7, 'cool': 0}",CR2y7yEm4X035ZMzrTtN9Q,Jim,5.0,6,user
1,"{'funny': 0, 'useful': 1, 'cool': 0}",_9GXoHhdxc30ujPaQwh6Ew,Kelle,1.0,2,user
2,"{'funny': 0, 'useful': 1, 'cool': 0}",8mM-nqxjg6pT04kwcjMbsw,Stephanie,5.0,2,user
3,"{'funny': 0, 'useful': 2, 'cool': 0}",Ch6CdTR2IVaVANr-RglMOg,T,5.0,2,user
4,"{'funny': 0, 'useful': 0, 'cool': 0}",NZrLmHRyiHmyT1JrfzkCOA,Beth,1.0,1,user


In [6]:
#list of all the business types
business['type'].unique()
#


array(['business'], dtype=object)

In [3]:
#merge dataframes into one
merged = pd.merge(review, business, on='business_id', how='left')
merged = pd.merge(merged, user, on='user_id', how='left')
merged = pd.merge(merged, checkin, on='business_id', how='left')

/var/folders/nd/hjy37y_d24d_62qk40rb90400000gn/T/ipykernel_57491/2199439912.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'type_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged = pd.merge(merged, checkin, on='business_id', how='left')


In [15]:
#show the name of all columns
merged.columns

Index(['votes_x', 'user_id', 'review_id', 'stars_x', 'date', 'text', 'type_x',
       'business_id', 'full_address', 'open', 'categories', 'city',
       'review_count_x', 'name_x', 'neighborhoods', 'longitude', 'state',
       'stars_y', 'latitude', 'type_y', 'votes_y', 'name_y', 'average_stars',
       'review_count_y', 'type_x', 'checkin_info', 'type_y'],
      dtype='object')

In [16]:
# List column name tuples that are similar
column_name_tuples = [(col, merged.columns.tolist().count(col)) for col in merged.columns]
column_name_tuples = list(set([tup for tup in column_name_tuples if tup[1] > 1]))

column_name_tuples

[('type_x', 2), ('type_y', 2)]